In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from contextlib import closing
import json

IS_FOR_PAPER = True
LOAD_FROM_JSON = True
AGG_RESULTS_JSON_FILE = '../results/services-range/agg_results.json'
ALL_RESULTS_JSON_FILE = '../results/services-range/all_results.json'
SAVE_TO_JSON = False
DATASET = 'inmates_enriched_10k'
METRIC_NAME = 'qualitative'
EXP_NUMBER = 4

In [ ]:
if not LOAD_FROM_JSON:
    '''
    with closing(sqlite3.connect("../db/remote-simulations-sadegh.db")) as connection:
        with closing(connection.cursor()) as cursor:
            rows = cursor.execute("SELECT * from results").fetchall()
            print(rows)
    '''
    with closing(sqlite3.connect("../db/remote-simulations-sadegh.db")) as connection_sadegh, \
            closing(sqlite3.connect("../db/remote-simulations.db")) as connection_up, \
            closing(sqlite3.connect("../db/remote-simulations-vmware.db")) as connection_vmware:
        sql_query = pd.read_sql_query("SELECT * from results", connection_sadegh)
        results_sadegh = pd.DataFrame(sql_query)
        sql_query = pd.read_sql_query("SELECT * from results", connection_up)
        results_up = pd.DataFrame(sql_query)
        sql_query = pd.read_sql_query("SELECT * from results", connection_vmware)
        results_vmware = pd.DataFrame(sql_query)
    
    results = pd.concat([results_sadegh, results_up, results_vmware])

In [ ]:
def invert(metric_values):
    return list([1 - x for x in metric_values])

def normalize(metric_values):
    values = invert(metric_values)
    # the value becoming 1
    highest = None if len(values) == 0 else values[-1]
    
    return list([x / highest for x in values])

In [ ]:
# different quality metrics
exp_params = {
    1: {
        'NODES_COUNT': 7,
        'SERVICES_COUNT': 5,
        'METRIC_NAME': METRIC_NAME,
        'ROW_LOWER_BOUND': 0.2,
        'ROW_UPPER_BOUND': 1,
        'COLUMN_LOWER_BOUND': 0.6,
        'COLUMN_UPPER_BOUND': 0.9,
        'FILTERING_TYPE': 'mixed'
    },
    2: {
        'NODES_COUNT': 9,
        'SERVICES_COUNT': 5,
        'METRIC_NAME': METRIC_NAME,
        'ROW_LOWER_BOUND': 0.2,
        'ROW_UPPER_BOUND': 1,
        'COLUMN_LOWER_BOUND': 0.6,
        'COLUMN_UPPER_BOUND': 0.9,
        'FILTERING_TYPE': 'mixed'
    },
    3: {
        'NODES_COUNT': 7,
        'SERVICES_COUNT': 5,
        'METRIC_NAME': 'qualitative',
        'DATASET': DATASET,
        'FILTERING_TYPE': 'mixed'
    },
    4: {
        'NODES_COUNT': 6,
        'METRIC_NAME': 'qualitative',
        'ROW_LOWER_BOUND': 0.2,
        'ROW_UPPER_BOUND': 1,
        'COLUMN_LOWER_BOUND': 0.6,
        'COLUMN_UPPER_BOUND': 0.9,
        'DATASET': 'inmates_enriched_10k',
        'FILTERING_TYPE': 'mixed'
    }
}

exp_options = {
    1: {
        "compare_props": ['dataset'],
        'filters': [
            lambda df: df['services_count'] == exp_params[1]['SERVICES_COUNT'],
            lambda df: df['nodes_count'] == exp_params[1]['NODES_COUNT'],
            lambda df: df['metric_name'] == exp_params[1]['METRIC_NAME'],
            lambda df: df['row_lower_bound'] == exp_params[1]['ROW_LOWER_BOUND'],
            lambda df: df['row_upper_bound'] == exp_params[1]['ROW_UPPER_BOUND'],
            lambda df: df['column_lower_bound'] == exp_params[1]['COLUMN_LOWER_BOUND'],
            lambda df: df['column_upper_bound'] == exp_params[1]['COLUMN_UPPER_BOUND'],
            lambda df: df['filtering_type'] == exp_params[1]['FILTERING_TYPE'],
        ],
        'legenda': lambda prop_values, i: str(prop_values)[7:-1],
        'metric_transformation': normalize
    },
    2: {
        "compare_props": ['dataset'],
        'filters': [
            lambda df: df['services_count'] == exp_params[2]['SERVICES_COUNT'],
            lambda df: df['nodes_count'] == exp_params[2]['NODES_COUNT'],
            lambda df: df['metric_name'] == exp_params[2]['METRIC_NAME'],
            lambda df: df['row_lower_bound'] == exp_params[2]['ROW_LOWER_BOUND'],
            lambda df: df['row_upper_bound'] == exp_params[2]['ROW_UPPER_BOUND'],
            lambda df: df['column_lower_bound'] == exp_params[2]['COLUMN_LOWER_BOUND'],
            lambda df: df['column_upper_bound'] == exp_params[2]['COLUMN_UPPER_BOUND'],
            lambda df: df['filtering_type'] == exp_params[2]['FILTERING_TYPE'],
        ],
        'legenda': lambda prop_values, i: str(prop_values)[7:-1],
        'metric_transformation': normalize
    },
    3: {
        "compare_props": ['row_lower_bound', 'row_upper_bound', 'column_lower_bound', 'column_upper_bound'],
        'filters': [
            lambda df: df['services_count'] == exp_params[3]['SERVICES_COUNT'],
            lambda df: df['nodes_count'] == exp_params[3]['NODES_COUNT'],
            lambda df: df['metric_name'] == exp_params[3]['METRIC_NAME'],
            lambda df: df['dataset'] == exp_params[3]['DATASET'],
            lambda df: df['filtering_type'] == exp_params[3]['FILTERING_TYPE'],
        ],
        'legenda': lambda prop_values, i: chr(ord('A') + i),
        'metric_transformation': invert
    },
    4: {
        "compare_props": ['services_count'],
        'filters': [
            lambda df: df['nodes_count'] == exp_params[4]['NODES_COUNT'],
            lambda df: df['metric_name'] == exp_params[4]['METRIC_NAME'],
            lambda df: df['row_lower_bound'] == exp_params[4]['ROW_LOWER_BOUND'],
            lambda df: df['row_upper_bound'] == exp_params[4]['ROW_UPPER_BOUND'],
            lambda df: df['column_lower_bound'] == exp_params[4]['COLUMN_LOWER_BOUND'],
            lambda df: df['column_upper_bound'] == exp_params[4]['COLUMN_UPPER_BOUND'],
            lambda df: df['dataset'] == exp_params[4]['DATASET'],
            lambda df: df['filtering_type'] == exp_params[4]['FILTERING_TYPE'],
        ],
        'legenda': lambda prop_values, i: 'S=' + str(prop_values),
        'metric_transformation': invert
    }
}

In [ ]:
# import operator
experiment_params  = exp_params[EXP_NUMBER]
experiment_options = exp_options[EXP_NUMBER]
COMPARE_PROPS = experiment_options['compare_props']
params = experiment_params

if not LOAD_FROM_JSON:
    import functools
        
    props_values = results[COMPARE_PROPS].value_counts().index.to_frame(index=False)
    
    results_filters = experiment_options['filters']
    
    def get_compare_props_filters(p_values):
        result = []
        for field in p_values._fields:
            field_value = getattr(p_values, field)
            result.append((lambda f, fv: lambda df: df[f] == fv)(field, field_value))
        
        return result
    
    def apply_filters_to_dataset(df, filters):
        compound_condition = functools.reduce(operator.and_, map(lambda filter: filter(df), filters))
        return df[compound_condition]
    
    props_values

## Aggregate results by multiple dimension

After filtering the results, they are grouped by `window_size` so that metric values with different `experiment_id` are averaged.

In [ ]:
agg_results = {}

if LOAD_FROM_JSON:
    with open(AGG_RESULTS_JSON_FILE, 'r') as f:
        agg_results = json.load(f)

    for p_values in agg_results:
        agg_results[p_values] = dict(zip(
            [int(k) for k in agg_results[p_values]['metric_value'].keys()],
            experiment_options['metric_transformation'](agg_results[p_values]['metric_value'].values())
        ))
else:    
    for p_values in props_values.itertuples(index=False):
        filtered_results = apply_filters_to_dataset(results, results_filters + get_compare_props_filters(p_values)) \
            .groupby(['window_size'])[['metric_value']].mean() \
            .sort_values(by=['window_size', 'metric_value'])
    
        display(filtered_results)
        agg_results[p_values] = filtered_results

In [ ]:
if SAVE_TO_JSON:
    def save_aggregated_results(agg_results):
        return { str(k): v.to_dict() for k, v in agg_results.items() }
        
    with open('saved_agg.json', 'w') as f:
        f.write(json.dumps(save_aggregated_results(agg_results)))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
# plt.figure(figsize=(10, 6))

for i, (p_values, dresults) in enumerate(sorted(map(lambda x: (int(x[0]), x[1]), agg_results.items()), key=lambda x: x[0])):
    if len(dresults) == 0:
        continue
    ax.plot(dresults.keys(), dresults.values(), label=experiment_options['legenda'](p_values, i))

plot_title = ', '.join(map(lambda x: f'{x[0]}={x[1]}', params.items()))
if not IS_FOR_PAPER:
    ax.set_title(plot_title)
ax.set_xlabel('Window size')
ax.set_ylabel('Metric value')
ax.grid(True)

if IS_FOR_PAPER:
    ax.legend(bbox_to_anchor=(1.1, 0.1), loc='lower center', borderaxespad=0)
else:
    ax.legend(bbox_to_anchor=(1.02, 0.1), loc='upper left', borderaxespad=0)

plt.savefig(f'agg_{",".join(COMPARE_PROPS)}.svg', bbox_inches = "tight")
plt.show()

## All experiments by Dataset

Results for the same experiment id are not aggregated but have their own plot

In [ ]:
results_map = {}

if LOAD_FROM_JSON:
    with open(ALL_RESULTS_JSON_FILE, 'r') as f:
        results_map = json.load(f)

    for p_values_map in results_map:
        for exp_id in results_map[p_values_map]:
            results_map[p_values_map][exp_id] = dict(zip(
                [x for x in results_map[p_values_map][exp_id]['window_size'].values()],
                experiment_options['metric_transformation']([x for x in results_map[p_values_map][exp_id]['metric_value'].values()])
            ))
else:
    for p_values in props_values.itertuples(index=False):
        results_map[p_values] = {}
        filtered_results = apply_filters_to_dataset(results, results_filters + get_compare_props_filters(p_values))
        experiment_ids = filtered_results['experiment_id'].unique()
        for exp_id in experiment_ids:
            exp_id_results = filtered_results[filtered_results['experiment_id'] == exp_id][['window_size', 'metric_value']].drop_duplicates()
            results_map[p_values][exp_id] = exp_id_results

In [ ]:
from typing import Dict, List, Tuple
import functools

def compute_diff_for_results_per_prop_reducer(acc: List[Tuple[float, float]], cur: float):
    '''
    acc: (metric difference with the previous window size, metric for the current window size)
    '''
    metric_diff = cur - acc[-1][1]
    return acc + [(metric_diff, cur)]

def compute_diff_for_results_per_prop(results_per_prop: Dict[str, Dict[int, float]]) -> List[List[float]]:
    return np.array([
        np.array(list(map(lambda x: x[0], functools.reduce(compute_diff_for_results_per_prop_reducer, \
                         list(exp_results.values())[1:], \
                         [(0, list(exp_results.values())[0])] \
                        )))) \
        for exp_results in results_per_prop.values()
    ])

In [ ]:
if SAVE_TO_JSON:
    def save_all_results(all_results):
        return { str(k): { str(k1): v1.to_dict() for k1, v1 in v.items() } for k, v in all_results.items() }
        
    with open('saved_all.json', 'w') as f:
        f.write(json.dumps(save_all_results(results_map)))

In [ ]:
import matplotlib.pyplot as plt

def set_plot_settings(ax, title):
    ax.set_xlabel('Window size')
    ax.set_ylabel('Metric value')
    ax.set_title(title)
    ax.grid(True)
    

results_map = {k:v for k, v in results_map.items() if len(v) > 0}
# 1. a boxplot
# 2. a boxplot where elements are the difference between the metric of winsize `x` and winsize `x - 1`
# 3. one plot that shows all the plot in the same subfigure
additional_plot_count = 3
fig_width = 9 if IS_FOR_PAPER else (max(len(exp_res) + additional_plot_count for exp_res in results_map.values()) * 4)

results_count = len(results_map)
fig_height = results_count * 5
fig = plt.figure(figsize=(fig_width, fig_height), constrained_layout=True)

plot_title = ', '.join(map(lambda x: f'{x[0]}={x[1]}', params.items()))
if not IS_FOR_PAPER:
    fig.suptitle(plot_title)

subfigs = fig.subfigures(nrows=results_count, ncols=1, squeeze=False)

for i, (p_values, results_per_prop) in enumerate(sorted(results_map.items(), key=lambda x: x[0])):
    subfig = subfigs.item(i)
    subfig.suptitle(f'Results for {str(p_values)[6:]}')

    axs_len = 2 if IS_FOR_PAPER else (len(results_per_prop) + additional_plot_count)
    axs = subfig.subplots(1, axs_len, sharex=True, squeeze=True)
    min_y = min(min(results_per_prop[k].values()) for k in results_per_prop.keys())
    max_y = max(max(results_per_prop[k].values()) for k in results_per_prop.keys())
    # this offset let the plot to be slightly distant from the figure border
    y_offset = (max_y - min_y) / 15
    for i, ax in enumerate(axs):
        if i != 1:
            ax.set_ylim(min_y - y_offset, max_y + y_offset)

    # additional_plot 1
    axs[0].boxplot([[results_per_prop[exp_id][win_size] for exp_id in results_per_prop.keys()] 
                   for win_size in list(results_per_prop.items())[0][1].keys()])
    prop_agg_results = agg_results[p_values]
    axs[0].plot(list(prop_agg_results.keys()), list(prop_agg_results.values()), label=str(p_values)[7:-1])
    set_plot_settings(axs[0], f'Boxplot with average')

    # additional_plot 2
    axs[1].boxplot(compute_diff_for_results_per_prop(results_per_prop))
    set_plot_settings(axs[1], f'Boxplot for metric difference between winsizes')

    if not IS_FOR_PAPER:
        for exp_index, (exp_id, exp_id_results) in enumerate(results_per_prop.items()):
            # additional_plot 3
            axs[2].plot(exp_id_results['window_size'], exp_id_results['metric_value'])
            set_plot_settings(axs[2], f'All experiments')
            axs[exp_index + additional_plot_count].plot(exp_id_results['window_size'], exp_id_results['metric_value'])
            set_plot_settings(axs[exp_index + additional_plot_count], f'Experiment id = {exp_id}')

#fig.tight_layout()
plt.savefig(f'all_exp_{",".join(COMPARE_PROPS)}.svg', bbox_inches='tight')
plt.show()